## for embedding (dont run multiple times it will take time)


In [ ]:
## Download and unzip the embeddings if using for 1st time. Y
##You can also save and load from your specified path once you save in your folder

!wget http://nlp.stanford.edu/data/glove.twitter.27B.zip
!unzip -q glove.twitter.27B.zip

--2023-08-13 15:39:46--  http://nlp.stanford.edu/data/glove.twitter.27B.zip
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://nlp.stanford.edu/data/glove.twitter.27B.zip [following]
--2023-08-13 15:39:46--  https://nlp.stanford.edu/data/glove.twitter.27B.zip
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://downloads.cs.stanford.edu/nlp/data/glove.twitter.27B.zip [following]
--2023-08-13 15:39:46--  https://downloads.cs.stanford.edu/nlp/data/glove.twitter.27B.zip
Resolving downloads.cs.stanford.edu (downloads.cs.stanford.edu)... 171.64.64.22
Connecting to downloads.cs.stanford.edu (downloads.cs.stanford.edu)|171.64.64.22|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1520408563 (1.4G) [ap

In [ ]:
GLOVE_EMB_TRAINED = "glove_emb_crf_weights.pkl"
GLOVE_VECTORIZER_TRAINED = "glove_crf_tv_layer.pkl"
GLOVE_EMB=200

In [ ]:
import numpy as np
glove_embeddings_path='/content/glove.twitter.27B.100d.txt'
glove_embeddings_index={}
with open(glove_embeddings_path, 'r', encoding='utf-8') as f:
    for line in f:
        values = line.split()
        word = values[0]
        coefs = np.asarray(values[1:], dtype='float32')
        glove_embeddings_index[word] = coefs

In [ ]:
#!pip install numpy==1.19.5
#!pip uninstall tensorflow
#!pip install tensorflow==2.2.0
#!pip install tensorflow-addons==0.10.0
!pip install tensorflow-addons
!pip install sklearn scipy
!pip install seqeval

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 612.1/612.1 kB 6.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for sklearn: filename=sklearn-0.0.post7-py3-none-any.whl size=2951 sha256=9f5f12b9e4f8a2a3d90ad9438a84d6e542528bd9a18f6dff55823fe57cadb44a
  Stored in directory: /root/.cache/pip/wheels/c8/9c/85/72901eb50bc4bc6e3b2629378d172384ea3dfd19759c77fd2c
Successfully built sklearn
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 1.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for seqeval: filename=seqeval-1.2.2-py3-none-any.whl size=16161 sha256=fa502e6b6ee2940f9038ffc4a9dfdf2ffca306d6518257a80b35064c7237a727
  Stored in directory: /root/.cache/pip/wheels/1a/67/4a/ad4082dd7dfc30f2abfe4d80a2ed5926a506eb8a972b4767fa
Successfully built seqeval


In [ ]:
import random
import pickle
import json
import pandas as pd
import sys
import numpy as np
import tensorflow as tf
from tensorflow import keras
from sklearn.model_selection import train_test_split
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization
from tensorflow.keras.layers import Embedding
import os
from seqeval.metrics import precision_score, recall_score, f1_score, classification_report
from sklearn.model_selection import train_test_split

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install "/content/drive/MyDrive/hate rate/task 4/needed file/pytorch_crf-0.7.2-py3-none-any.whl"

Processing ./drive/MyDrive/hate rate/task 4/needed file/pytorch_crf-0.7.2-py3-none-any.whl


In [ ]:
from torchcrf import CRF

## part you can change

In [ ]:
#please upload the crf file before running the code
!cp /content/crf.py . # Done for colab
#!cp '/content/drive/MyDrive/hate rate/task 3/code (paper 1)/crf.py'
from crf import CRF as CRF_lib

cp: cannot stat '/content/crf.py': No such file or directory


In [ ]:
df=pd.read_csv('/content/drive/MyDrive/hate rate/task 3/dataset/train.csv',sep='|')


N_CRF_TAGS = 3
LSTM_UNITS = 512
DENSE_UNITS = 50
LSTM_DROPOUT = 0.2
DENSE_DROPOUT = 0.2
EPOCHS = 2 #(Default 5, check!)
BATCH_SIZE = 32
SEED = 42

GLOVE_EMB = 100  # Assuming you're using GloVe embeddings
GLOVE_EMBEDDING_DIM=100

MAX_LEN=128
# maximum word length is 112 and maximum tokenized size is 96 (because tokenizer will ignore punctuations of 112->96)


In [ ]:
df.head()

,Id,sentence,span,bio
0,0,"Say it loud , say it clear , illegal #immigran...","{'start': [8, 9], 'end': [8, 9]}",O O O O O O O O B I O O O O O
1,1,"Islam is not a religion, its just an excuse fo...","{'start': [0, 10], 'end': [4, 13]}",B I I I I O O O O O B I I I O O O O
2,2,@user Care About Illegals Breaking U.S #Immigr...,"{'start': [40], 'end': [40]}",O O O O O O O O O O O O O O O O O O O O O O O ...
3,3,Behold the future DACA America - unless we Man...,"{'start': [10, 17], 'end': [14, 18]}",O O O O O O O O O O B I I I I O O B I O O O O ...
4,4,No that little tattoo of a flower on your ankl...,"{'start': [14, 20, 25, 26], 'end': [14, 21, 25...",O O O O O O O O O O O O O O B O O O O O B I O ...


In [ ]:
df.columns

Index(['Id', 'sentence', 'span', 'bio'], dtype='object')

# code starts here

In [ ]:
maxx=0
sent=''
for s in df['sentence']:
  s_len=len(s.split())
  if(s_len>maxx):
    sent=s
    maxx=s_len





In [ ]:
print(maxx)
print(sent)

112
What I do know is that I live on a small Island and it seems that a lot of people from various countries in the world are willing to desert their homelands and their families just to come here , in my mind that tells me a lot about said people and their home countries , most of all it highlights their weaknesses of how their own countries have let them down and how they feel the need to feed off somebody elses country , how should we English interpret this without sounding rude ? ? ? , locust ? ? ? , parasite ? ? ? , you tell me . 


In [ ]:
tag2idx={'O':0,'B':1,"I":2}

In [ ]:
new_bio=pd.Series()
for indx,data in df.iterrows():
  arr=data['bio'].split()
  arr=[tag2idx[x] for x in arr]
  new_bio.at[indx]=arr


<ipython-input-16-e14f3cbb4249>:1: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  new_bio=pd.Series()


In [ ]:
df['new_bio']=new_bio

In [ ]:
train_df,test_df=train_test_split(df,test_size=0.2,random_state=50)

In [ ]:
#for training data
train_y = keras.preprocessing.sequence.pad_sequences(maxlen=MAX_LEN,
                                               sequences=train_df['new_bio'],
                                               padding="post",
                                               value=0)
#train_y = [keras.utils.to_categorical(i, num_classes=N_CRF_TAGS) for i in train_y]
train_y=np.asarray(train_y)

# for test data
test_y = keras.preprocessing.sequence.pad_sequences(maxlen=MAX_LEN,
                                               sequences=test_df['new_bio'],
                                               padding="post",
                                               value=0)
#test_y = [keras.utils.to_categorical(i, num_classes=N_CRF_TAGS) for i in test_y]
test_y=np.asarray(test_y)



In [ ]:
train_y.shape

(1936, 128)

In [ ]:
for x in train_y[1]:
  if(x==1):
    print('yes')

yes
yes
yes


In [ ]:
GLOVE_EMBEDDING_DIM=100

In [ ]:
import nltk
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [ ]:
# Step 1: Fit the tokenizer on the sentence corpus
tokenizer = Tokenizer(filters='',oov_token='[OOV]')

In [ ]:
GLOVE_EMB=100

In [ ]:


tokenizer.fit_on_texts(train_df['sentence'].tolist())
train_sequence=tokenizer.texts_to_sequences(train_df['sentence'].values)
test_sequence=tokenizer.texts_to_sequences(test_df['sentence'].values)


num_tokens=len(tokenizer.word_index)

num_tokens=len(tokenizer.word_index)+1

GLOVE_EMBEDDING_DIM=100





In [ ]:
oov_count=0
not_oov=0

#for train data
from pandas._libs.lib import is_float_array
embedding_matrix=np.zeros((num_tokens,GLOVE_EMBEDDING_DIM))
for word,i in tokenizer.word_index.items():
  embedding_vector=glove_embeddings_index.get(word)
  if embedding_vector is not None:
     not_oov+=1
     embedding_matrix[i]=embedding_vector
  else:
        oov_count+=1
        # Words not found in embedding index will be have random embedding.
        # This includes the representation for "padding" and "OOV"
        random_num1 = np.random.rand(GLOVE_EMB)
        random_num2 = np.random.rand(GLOVE_EMB)
        embedding_vector = [
            r1 if r2 < 0.5 else -1 * r1
            for r1, r2 in zip(random_num1, random_num2)
        ]
        # print(embedding_vector)
        embedding_matrix[i] = embedding_vector
print('the number of words out of oov ',oov_count)
print('number of words in vocabulary is ',not_oov)





the number of words out of oov  1393
number of words in vocabulary is  5645


In [ ]:
maxx_tokenized=0
for i in range(len(train_sequence)):
  current=len(train_sequence[i])
  maxx_tokenized=max(maxx_tokenized,current)

In [ ]:
maxx_tokenized

112

In [ ]:
train_maxx=0
sent=''
for s in train_df['sentence']:
  s_len=len(s.split())
  if(s_len>train_maxx):
    sent=s
    train_maxx=s_len

print(sent)
print(len(sent.split()))

What I do know is that I live on a small Island and it seems that a lot of people from various countries in the world are willing to desert their homelands and their families just to come here , in my mind that tells me a lot about said people and their home countries , most of all it highlights their weaknesses of how their own countries have let them down and how they feel the need to feed off somebody elses country , how should we English interpret this without sounding rude ? ? ? , locust ? ? ? , parasite ? ? ? , you tell me . 
112


so this time the number of token is same as the number of word/punctuation/speical

In [ ]:

train_padded_sequences = pad_sequences(train_sequence,padding='post', maxlen=MAX_LEN)
test_padded_sequences = pad_sequences(test_sequence,padding='post', maxlen=MAX_LEN)


In [ ]:
#checking padding index value
a=train_padded_sequences[:2]
b=a!=0
print(b)

[[ True  True  True  True  True  True  True  True  True  True  True  True
   True  True  True False False False False False False False False False
  False False False False False False False False False False False False
  False False False False False False False False False False False False
  False False False False False False False False False False False False
  False False False False False False False False False False False False
  False False False False False False False False False False False False
  False False False False False False False False False False False False
  False False False False False False False False False False False False
  False False False False False False False False False False False False
  False False False False False False False False]
 [ True  True  True  True  True  True  True  True  True  True  True  True
   True  True  True  True  True  True  True  True  True  True  True False
  False False False False False False False False False False

In [ ]:
train_y.shape

(1936, 128)

In [ ]:
train_padded_sequences.shape

(1936, 128)

In [ ]:
MAX_LEN

128

In [ ]:
import torch

In [ ]:
embedding_matrix=torch.tensor(embedding_matrix)

In [ ]:
# #USING TORCH
# import torch
# import torch.nn as nn
# from torchcrf import CRF

# # Assuming you have defined MAX_LEN, GLOVE_EMB, embedding_matrix, LSTM_UNITS, LSTM_DROPOUT, DENSE_UNITS, N_CRF_TAGS

# # Define the model architecture
# class our_model(nn.Module):
#     def __init__(self, num_tokens, num_tags, embedding_dim, lstm_units, lstm_dropout, dense_units, embedding_matrix):
#         super(our_model, self).__init__()
#         self.embedding = nn.Embedding.from_pretrained(torch.tensor(embedding_matrix, dtype=torch.float))
#         self.bilstm1 = nn.LSTM(input_size=embedding_dim, hidden_size=lstm_units, batch_first=True, bidirectional=True)
#         self.bilstm2 = nn.LSTM(input_size=embedding_dim, hidden_size=lstm_units, batch_first=True, bidirectional=True)
#         self.dense1 = nn.Linear(2 * lstm_units, dense_units)
#         self.dense2 = nn.Linear(dense_units, num_tags)
#         self.crf = CRF(num_tags)

#     def forward(self, input_text,tags):
#         input_text=input_text.to(device)
#         tags=tags.to(device)
#         mask = input_text != 0
#         mask = mask.permute(1, 0)
#         # PAD_TOKEN=0
#         # mask = torch.ne(input_text, PAD_TOKEN)
#         print(mask.shape)
#         x = self.embedding(input_text)
#         x1, _ = self.bilstm1(x)
#         x2, _ = self.bilstm2(x)
#         x = x1 + x2  # residual connection to the first biLSTM
#         x = self.dense1(x)
#         x = self.dense2(x)
#         x=x.permute(1,0,2)

#         out = self.crf.decode(x,tags,mask=mask)
#         print('out is ',out)
#         return out

# # Instantiate the model with the provided embedding_matrix
# num_tokens = len(embedding_matrix)
# num_tags = N_CRF_TAGS
# embedding_dim = GLOVE_EMB
# lstm_units = LSTM_UNITS
# lstm_dropout = LSTM_DROPOUT
# dense_units = DENSE_UNITS




In [ ]:
#USING TORCH
import torch
import torch.nn as nn
from torchcrf import CRF

# Assuming you have defined MAX_LEN, GLOVE_EMB, embedding_matrix, LSTM_UNITS, LSTM_DROPOUT, DENSE_UNITS, N_CRF_TAGS

# Define the model architecture
class our_model(nn.Module):
    def __init__(self, num_tokens, num_tags, embedding_dim, lstm_units, lstm_dropout, dense_units, embedding_matrix):
        super(our_model, self).__init__()
        self.embedding = nn.Embedding.from_pretrained(torch.tensor(embedding_matrix, dtype=torch.float))
        self.bilstm1 = nn.LSTM(input_size=embedding_dim, hidden_size=lstm_units, batch_first=True, bidirectional=True)
        self.bilstm2 = nn.LSTM(input_size=embedding_dim, hidden_size=lstm_units, batch_first=True, bidirectional=True)
        self.dense1 = nn.Linear(2 * lstm_units, dense_units)
        self.dense2 = nn.Linear(dense_units, num_tags)
        self.crf = CRF(num_tags)

    def forward(self, input_text,tags):
        input_text=input_text.to(device)
        if(tags is not None):
          tags=tags.to(device)
        x = self.embedding(input_text)
        x1, _ = self.bilstm1(x)
        print()
        x2, _ = self.bilstm2(x)
        x = x1 + x2  # residual connection to the first biLSTM
        x = self.dense1(x)
        x = self.dense2(x)
        x=x.permute(1,0,2)
        print('here')
        if tags is not None:
            print('inside if')
            out = self.crf(x, tags)
            return out
        else:
            print('have tags')
            # During inference, use viterbi_decode to get predicted tags
            x = x.permute(1, 0, 2)  # Transpose for CRF layer
            predicted_tags = self.crf.decode(x)
            return predicted_tags
        return out

# Instantiate the model with the provided embedding_matrix
num_tokens = len(embedding_matrix)
num_tags = N_CRF_TAGS
embedding_dim = GLOVE_EMB
lstm_units = LSTM_UNITS
lstm_dropout = LSTM_DROPOUT
dense_units = DENSE_UNITS




In [ ]:
# Check if a GPU device is available, otherwise use CPU
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
import torch
from torch.utils.data import Dataset, DataLoader

# Assuming you have tokenized and converted text data as a list of token indices
# Example: tokenized_data = [[2, 45, 6, 32], [10, 8, 20, 0], [15, 7, 9, 3, 1]]

class CustomDataset(Dataset):
    def __init__(self, tokenized_data, targets):
        self.data = tokenized_data
        self.targets = targets

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        return self.data[idx], self.targets[idx]

# Sample targets (labels), assuming you have them in a separate list or array
# Example: targets = [1, 0, 1]

# Assuming you already have padded sequences for train_y and targets
# Convert them to PyTorch tensors
train_input = torch.tensor(train_padded_sequences, dtype=torch.long)
targets = torch.tensor(train_y, dtype=torch.long)

# Define your DataLoader
batch_size = 32  # Adjust this based on your available memory and batch size preference
train_dataset = CustomDataset(train_input, targets)
train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)



In [ ]:
import torch
import torch.optim as optim
import torch.optim.lr_scheduler as lr_scheduler
import torch


# Load the model architecture
# Load the model architecture
# model_architecture_path = "/content/drive/MyDrive/Research/hate speech/model weights/bert_same_lstm_two_dense_two_mbert__epoch_40/model_architecture.pth"

model = our_model(num_tokens, num_tags, embedding_dim, lstm_units, lstm_dropout, dense_units, embedding_matrix)


# # # Load the model weights
#
# if(want_to_use_trained_model_which_is_saved_in_drive_for_this_code):
#   model_weights_path = where_is_the_model_saved_that_model_weight_file_location
#   model.load_state_dict(torch.load(model_weights_path))
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)


<ipython-input-122-6b4394b0f5d9>:12: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  self.embedding = nn.Embedding.from_pretrained(torch.tensor(embedding_matrix, dtype=torch.float))


our_model(
  (embedding): Embedding(7039, 100)
  (bilstm1): LSTM(100, 512, batch_first=True, bidirectional=True)
  (bilstm2): LSTM(100, 512, batch_first=True, bidirectional=True)
  (dense1): Linear(in_features=1024, out_features=50, bias=True)
  (dense2): Linear(in_features=50, out_features=3, bias=True)
  (crf): CRF(num_tags=3)
)

In [ ]:
import torch.nn.functional as F
from torch.optim.lr_scheduler import ReduceLROnPlateau

# Train the model
optimizer = optim.AdamW(model.parameters(), lr=0.1, weight_decay=0.001)
lr_scheduler = ReduceLROnPlateau(optimizer, mode='min', factor=0.7, patience=1, verbose=True)
model.train()

#model training
n_epochs = 2
loss_fn = nn.BCELoss()
total_data=len(train_df)
prev_total_loss = None  # Variable to store the previous total_loss
best_accuracy=0
for epoch in range(n_epochs):
    total_loss = 0
    total_samples_train = 0
    batch_c = 0
    correct_predictions_train = 0
    optimizer.zero_grad()

    for batch_inputs, batch_targets in train_dataloader:
        batch_c += 1

        batch_inputs = batch_inputs.long()
        #y_true = batch['label'].unsqueeze(1).float().to(device)
        # Transpose the logits to the shape (seq_length, batch_size, num_tags)
        tags = batch_targets.permute(1, 0)
        y_pred = model(batch_inputs,tags)

        loss = loss_fn(y_pred, y_true)
        total_loss += loss.item()
        optimizer.zero_grad()
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
        optimizer.step()

        predicted_labels_train = (y_pred >= threshold).float()

        total_samples_train += len(y_true)
        for j in range(len(predicted_labels_train)):
            if y_true[j] == predicted_labels_train[j]:
                correct_predictions_train += 1

    total_loss /= total_data

    # # Check if the total_loss has changed by less than 2
    # if prev_total_loss is not None and abs(total_loss - prev_total_loss) < 0.005:
    #     lr_scheduler.step(total_loss)


    prev_total_loss = total_loss

    correct_predictions_val = 0
    total_samples_val = 0

    for val_batch in val_data_loader:
        text_input = val_batch['text_input']

        y_true = val_batch['label'].unsqueeze(1).float().to(device)
        y_pred = model(text_input)

        predicted_labels = (y_pred >= threshold).float()
        total_samples_val += len(y_true)

        for j in range(len(predicted_labels)):
            if y_true[j] == predicted_labels[j]:
                correct_predictions_val += 1
    print("**********************")
    print('Epoch:', epoch)
    validation_accuracy=(correct_predictions_val / total_samples_val) * 100
    val_accuracy=(correct_predictions_val / total_samples_val) * 100
    print('Val Accuracy:', val_accuracy)
    train_accuracy=(correct_predictions_train / total_samples_train) * 100
    print('Train Accuracy:', train_accuracy)
    learning_rate=optimizer.param_groups[0]['lr']
    print('Current Learning Rate:', learning_rate)
    print('Total Loss:', total_loss)

    new_row={'epoch':epoch,'val accuracy':val_accuracy,'train accuracy':train_accuracy,'loss':total_loss,'learning rate':learning_rate}
    model_stats=model_stats.append(new_row,ignore_index=True)
    if ( validation_accuracy > best_accuracy and (want_to_save_trained_model == True)) :
        best_accuracy = validation_accuracy
        torch.save(model.state_dict(), location1)
        print(f"Saved the best model with validation accuracy: {best_accuracy:.2f}")
        torch.save(model, location2)




here
inside if
y pred is  tensor(-4676.9229, device='cuda:0', grad_fn=<SumBackward0>)

here
inside if
y pred is  tensor(-4681.5674, device='cuda:0', grad_fn=<SumBackward0>)

here
inside if
y pred is  tensor(-4679.0029, device='cuda:0', grad_fn=<SumBackward0>)

here
inside if
y pred is  tensor(-4679.3965, device='cuda:0', grad_fn=<SumBackward0>)

here
inside if
y pred is  tensor(-4674.1465, device='cuda:0', grad_fn=<SumBackward0>)

here
inside if
y pred is  tensor(-4677.8276, device='cuda:0', grad_fn=<SumBackward0>)

here
inside if
y pred is  tensor(-4674.5688, device='cuda:0', grad_fn=<SumBackward0>)

here
inside if
y pred is  tensor(-4676.0195, device='cuda:0', grad_fn=<SumBackward0>)

here
inside if
y pred is  tensor(-4682.2148, device='cuda:0', grad_fn=<SumBackward0>)

here
inside if
y pred is  tensor(-4682.3330, device='cuda:0', grad_fn=<SumBackward0>)

here
inside if
y pred is  tensor(-4675.5107, device='cuda:0', grad_fn=<SumBackward0>)

here
inside if
y pred is  tensor(-4680.605

KeyboardInterrupt: ignored

to do : 1. cuda
2. testing and validation

In [ ]:
import torch.optim as optim




# Define the optimizer and other settings
optimizer = optim.AdamW(model.parameters(), lr=0.1, weight_decay=0.001)
# Define other hyperparameters and settings for training

# Training loop
for epoch in range(n_epochs):
    model.train()  # Set the model to training mode

    total_loss = 0
    total_samples_train = 0
    correct_predictions_train = 0

    for batch_inputs, batch_targets in train_dataloader:
        batch_inputs = batch_inputs.long()

        # Transpose the tags to the shape (seq_length, batch_size)
        tags = batch_targets.permute(1, 0)

        # Forward pass: Compute the CRF negative log-likelihood loss
        out = model(batch_inputs, tags)

        # Compute the mean loss value across the batch
        loss = out.mean()

        # Backward pass: Compute gradients and update model parameters
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        total_loss += loss.item()
        total_samples_train += batch_inputs.size(0)  # Batch size
        # Calculate accuracy or other metrics if needed

    # Compute and print average loss for the epoch
    avg_loss = total_loss / len(train_dataloader)
    print(f"Epoch {epoch + 1}, Average Loss: {avg_loss:.4f}")

# Once training is done, you can use the model for inference and prediction as before


In [ ]:
idx2tag = {i: w for w, i in tag2idx.items()}

In [ ]:
# Get the model's predictions on the test data
predictions = model.predict(test_padded_sequences)

16/16 [==============================] - 4s 256ms/step


In [ ]:
predictions.shape
predicted_classes = tf.argmax(predictions, axis=2)

In [ ]:
predicted_classes

<tf.Tensor: shape=(485, 128), dtype=int64, numpy=
array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 1, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])>

In [ ]:
test_y.shape
true_classes = tf.argmax(test_y, axis=2)

In [ ]:




# Calculate the accuracy manually using a loop
total_samples = len(test_y)
correct_predictions = 0
# Calculate the accuracy for each token for all samples
accuracies_per_token = np.mean(predicted_classes == true_classes, axis=0)

# Calculate the average accuracy across all tokens
average_accuracy = np.mean(accuracies_per_token)

print(average_accuracy)


0.9678157216494846


In [ ]:
# X_test_tensor = tf.convert_to_tensor(X_test)
# y_test_tensor = tf.convert_to_tensor(y_test)

# Evaluate the model on the test dataset
results = model.evaluate(test_padded_sequences, test_y)
test_accuracy = results[1]
print(f"Test Accuracy: {test_accuracy:.2f}")

16/16 [==============================] - 3s 170ms/step - loss: 0.1060 - accuracy: 0.9678
Test Accuracy: 0.97


In [ ]:
import nltk
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [ ]:
# Step 1: Fit the tokenizer on the sentence corpus
tokenizer = Tokenizer(filters='')

In [ ]:


# Sample sentences
sentences = ["hey how are you hi there "]


#tokenizer.fit_on_texts(sentences)

# Step 2: Tokenization (word and punctuation)
out = tokenizer.texts_to_sequences(sentences)

In [ ]:
len(out[0])

0

In [ ]:
train_sent=train_df['sentence'].tolist()

In [ ]:
type(train_sent)

list

In [ ]:
tokenizer=Tokenizer(filters=' ')

In [ ]:
len(train_sent)

1936

In [ ]:
a=train_sent[0]

In [ ]:
tokenizer.fit_on_texts(a)

In [ ]:
type(a)

str

In [ ]:
a

'Are there any whites left in that craphole or have they all been killed ?'

In [ ]:
a=f"{a}"
b=["Are there any whites left in that craphole or have they all been killed ?"]

In [ ]:
a_len=a.split()
print(a_len)
print(len(a_len))

['Are', 'there', 'any', 'whites', 'left', 'in', 'that', 'craphole', 'or', 'have', 'they', 'all', 'been', 'killed', '?']
15


In [ ]:
b

['Are there any whites left in that craphole or have they all been killed ?']

In [ ]:
outputs=tokenizer.texts_to_sequences("Are there any whites left in that craphole or have they all been killed ?")


so if you input array then it works fine

In [ ]:
outputs

[[2],
 [6],
 [1],
 [],
 [3],
 [4],
 [1],
 [6],
 [1],
 [],
 [2],
 [7],
 [9],
 [],
 [11],
 [4],
 [8],
 [3],
 [1],
 [12],
 [],
 [5],
 [1],
 [13],
 [3],
 [],
 [8],
 [7],
 [],
 [3],
 [4],
 [2],
 [3],
 [],
 [14],
 [6],
 [2],
 [15],
 [4],
 [10],
 [5],
 [1],
 [],
 [10],
 [6],
 [],
 [4],
 [2],
 [16],
 [1],
 [],
 [3],
 [4],
 [1],
 [9],
 [],
 [2],
 [5],
 [5],
 [],
 [17],
 [1],
 [1],
 [7],
 [],
 [18],
 [8],
 [5],
 [5],
 [1],
 [19],
 [],
 [20]]

In [ ]:
len(outputs)

73

In [ ]:
import tensorflow as tf

# Create a TensorFlow Tokenizer
tokenizer = tf.keras.preprocessing.text.Tokenizer(filters='',oov_token='[OOV]')

# Fit the tokenizer on the input text
tokenizer.fit_on_texts([a])

def custom_tokenizer(input_text):


    # Get the word index from the tokenizer
    word_index = tokenizer.word_index

    # Tokenize the input text using the tokenizer
    tokens = tokenizer.texts_to_sequences([input_text])[0]
    print(tokens)
    # Convert tokens back to words, punctuation, and special characters
    reverse_word_index = {v: k for k, v in word_index.items()}
    tokens = [reverse_word_index[token] for token in tokens]

    return tokens


tokens = custom_tokenizer(a)
print(tokens)


[2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16]
['are', 'there', 'any', 'whites', 'left', 'in', 'that', 'craphole', 'or', 'have', 'they', 'all', 'been', 'killed', '?']


In [ ]:
testing=pd.Series()


<ipython-input-116-5eabf16cdd78>:1: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  testing=pd.Series()


In [ ]:
testing.at[0]="hey whites left here"
testing.at[1]="are jsdklfjh in that craphole"

In [ ]:
testing.values

array(['hey whites left here', 'are jsdklfjh in that craphole'],
      dtype=object)

In [ ]:
output=tokenizer.texts_to_sequences(testing.values)

In [ ]:
output

[[1, 5, 6, 1], [2, 1, 7, 8, 9]]